In [2]:
import numpy as np
from scipy import linalg
from scipy.optimize import minimize

In [3]:
#                 (0,244)--------(294,244)
#                     |               |
# Polygone ->         |               |
#                     |               |
#                   (0,0)--------(294, 0)

In [56]:
pol1 = (582/1280, 271/720)
pol1_2 = (486/1280, 384/720)
pol2 = (371/1280, 520/720)
pol2_2 = (614/1280, 601/720)
pol3 = (945/1280, 710/720)
pol3_2 = (976/1280, 471/720)
pol4 = (994/1280, 317/720)
pol4_2 = (764/1280, 290/720)


x_res, y_res = (1920, 1080)# resolution
polygon = np.array([(pol1[0] * x_res, pol1[1] * y_res), 
                    (pol1_2[0] * x_res, pol1_2[1] * y_res), 
                    (pol2[0] * x_res, pol2[1] * y_res),
                    (pol2_2[0] * x_res, pol2_2[1] * y_res),
                    (pol3[0] * x_res, pol3[1] * y_res),
                    (pol3_2[0] * x_res, pol3_2[1] * y_res),
                    (pol4[0] * x_res, pol4[1] * y_res), 
                    (pol4_2[0] * x_res, pol4_2[1] * y_res)], dtype=int)#points for polygone in the center


def solveForCoefs(params):
    """
    REQUIRE:
    pointsW = [[xw,yw],[xw2,yw2],...,[xwN,ywN]] - WORLD COORDS
    pointsPX = [[u,v],[u2,v2],...,[uN,vN]] - IMAGE COORDS
    """
    fx, fy, cx, cy, phi1, phi2, phi3, tx, ty, tz= params

    pointsW = [(0, 244), (0,122), (0, 0), (147, 0), (294, 0), (294, 122), (294, 244), (147, 244)]
    pointsPX = polygon

    # внутр. матрица
    M_int = np.array([[fx, 0, cx, 0],
                     [0, fy, cy, 0],
                     [0, 0, 1, 0]])
    # 3 матрицы поворота
    m_rot1 = np.array([[1, 0, 0], [0, np.cos(phi1), -np.sin(phi1)], [0, np.sin(phi1), np.cos(phi1)]])
    m_rot2 = np.array([[np.cos(phi2), 0, -np.sin(phi2)], [0, 1, 0], [np.sin(phi2), 0, np.cos(phi2)]])
    m_rot3 = np.array([[np.cos(phi3), -np.sin(phi3), 0], [np.sin(phi3), np.cos(phi3), 0], [0, 0, 1]])
    m_rot = np.matmul(np.matmul(m_rot1, m_rot2), m_rot3) # произведение матриц поворота

    # расширение матрицы поворота (справа - вектором смещения, снизу стркой [0,0,0,1]). Теперь это внешняя матрица
    M_ext = np.vstack((np.hstack((m_rot, [[tx], [ty], [tz]])), [0, 0, 0, 1]))
    # произведение внутренней и вшешних матриц
    A = np.matmul(M_int, M_ext)

    # Задаем метрику кврадартичного отклонения
    metric = 0
    # Проходимся по всем world и внутренним точкам. Считаем сумму квадратичных отклонений
    for W, PX in zip(pointsW, pointsPX):
        X = np.append(W, [0, 1])
        y_pred = np.matmul(A, X)
        y_pred = y_pred / y_pred[-1]
        y_pred += [1920/2, 1080/2, 0]
        metric += np.sum((PX - y_pred[:2])**2)

    metric = metric / len(pointsW)
    print(metric)
    return metric


x0 = [0.5, 0.5, 0.5, 0.5, 5, 5, 5, 243, -120, 200]
# x0 = [2.8324707577253414,-1.9671619766483819,2.7862322829902353,2.391494243282522,4.12283312235628,6.944832512986978,4.8948156279340065,9.8628366966564,-168.8271237929663,206.37985786723056]
out = minimize(solveForCoefs, x0, method='SLSQP')
print(*out.x, sep=",")

191076.76451928518
191076.7645187943
191076.7645193053
191076.7645159074
191076.7645154622
191076.76451744934
191076.76451860226
191076.76451668472
191076.76451928553
191076.7645192823
191076.76451928547
257709.26271195742
165384.60389145237
165384.6038929504
165384.60389297668
165384.603890659
165384.60389038047
165384.6038860124
165384.60388873445
165384.6039221327
165384.6038915478
165384.60389145228
165384.60389130897
3183601.3468993334
209617.8239317755
159722.43599529317
159722.4359941507
159722.4359927569
159722.43599600092
159722.43599517376
159722.4355227971
159722.43604810876
159722.43576941392
159722.43599505458
159722.4359952882
159722.43599616375
4339206718248.7485
44441360731.48045
497833072.0913167
17645535.576886043
3171979.8297301535
784302.5194132784
311071.6494133925
196980.70296195292
206602.74528185907
159059.64357989645
142358.5299134559
142358.5299171266
142358.52992613456
142358.52991605992
142358.52991383144
142358.53006233607
142358.53008347642
142358.52938134

In [54]:
def getRealcoords(pt):
    fx, fy, cx, cy, phi1, phi2, phi3, tx, ty, tz = \
        [187.08826537858658,-256.73158416715137,49.6722205130153,980.7968744515073,75.53078665035575,31.395759410121656,62.44511493820521,-166.01045337973667,196.03770232314747,68.71317060268947]

    
    M_int = np.array([  [fx, 0, cx, 0],
                        [0, fy, cy, 0],
                        [0, 0, 1, 0]])

    m_rot1 = np.array([[1, 0, 0], [0, np.cos(phi1), -np.sin(phi1)], [0, np.sin(phi1), np.cos(phi1)]])
    m_rot2 = np.array([[np.cos(phi2), 0, -np.sin(phi2)], [0, 1, 0], [np.sin(phi2), 0, np.cos(phi2)]]) 
    m_rot3 = np.array([[np.cos(phi3), -np.sin(phi3), 0], [np.sin(phi3), np.cos(phi3), 0], [0, 0, 1]])
    m_rot = np.matmul(np.matmul(m_rot2, m_rot1), m_rot3)
    M_ext = np.vstack((np.hstack((m_rot, [[tx], [ty], [tz]])), [0, 0, 0, 1]))
    A = np.matmul(M_int, M_ext)

    b = np.array([pt[0], pt[1], 0])
    answ = np.linalg.lstsq(A,b, rcond=None)[0]
    answ = answ / answ[-1]
    return answ

    # b = np.array([pt[0], pt[1], 0, 1])
    # answ = np.matmul(A, b)
    # answ = answ / answ[-1]
    # answ += [1920/2, 1080/2, 0]
    # return answ[:2]




print(polygon)
print(getRealcoords([556, 780]))

[[ 873  406]
 [ 729  576]
 [ 556  780]
 [ 921  901]
 [1417 1065]
 [1464  706]
 [1491  475]
 [1146  435]]
[-20.70559299 -16.97597287 -68.80084745   1.        ]
